In [1]:
#テキストの属性をもとに段落を判別しつつテキストデータを取得
#ヘッダ・フッタ等をテキストブロックの属性情報（特に位置情報）で除外
#テキストを取得する範囲も、テキストブロックの開始位置の座標情報をもとに、強制的に読み込み順序を制御
#（リスト形式で、x軸の閾値を設定：ブランクのリストであればブロックの順番通りに読み込み）
#各ブロックの座標の分布は、Extract_Pdf_Attrib.ipynbで確認可能
import fitz  # PyMuPDF
import pandas as pd
import re
import sys
sys.path.append('/work_dir') 
import csv
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

# ログファイルの設定
log_filename = "data/logfile_Pdf2Txt.log"
logging.basicConfig(
    level=logging.INFO,  # ログレベルの設定 (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s - %(levelname)s - %(message)s",  # ログのフォーマット
    handlers=[
        logging.FileHandler(log_filename, mode="a", encoding="utf-8"),  # ファイルに記録
        logging.StreamHandler()  # コンソールにも出力
    ]
)


In [2]:
# CSVを読み込む関数
def load_csv(file_path):
    """
    CSVを読み込み、DataFrameを返す
    file_path: CSVファイルのパス
    """
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: CSV file not found at {file_path}")
        logging.error(f"Error: CSV file not found at {file_path}")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file at {file_path} is empty or invalid")
        logging.error(f"Error: CSV file at {file_path} is empty or invalid")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        logging.error(f"Error reading CSV file: {e}")
        return None

In [3]:
def clean_text_by_attributes(page, header_y, footer_y, left_margin_x, right_margin_x, column_thresholds, line_spacing_factor):
    text = ""
    blocks = page.get_text("dict")["blocks"]
    #header_y=None
    #footer_y=None
    #left_margin_x=None
    #right_margin_x=None
    #column_thresholds=None
    #line_spacing_factor=None

    def get_range_index(x0, ranges):
        """x0値がどの範囲に属するかを判定"""
        for i, bound in enumerate(ranges):
            if x0 < bound:
                return i
        return len(ranges)  # 範囲外は最後の区分

    # 各ブロックにLine1のbbox_x0を取得して分類
    sorted_blocks = {i: [] for i in range(len(column_thresholds) + 1)}  # 範囲ごとのリスト
    for block_index, block in enumerate(blocks):
        if "lines" not in block or not block["lines"]:
            continue
        # Line1のbbox_x0を取得
        line1_bbox_x0 = block["lines"][0]["spans"][0]["bbox"][0] if block["lines"][0]["spans"] else None
        if line1_bbox_x0 is not None:
            range_index = get_range_index(line1_bbox_x0, column_thresholds)
            sorted_blocks[range_index].append((block_index, block))  # block_indexを保持

    # 各範囲ごとのブロックを番号順に処理
    previous_font = None
    previous_size = None
    previous_color = None
    previous_bbox = None
    previous_text_type = None

    #line_spacing_factorがなければ、とりあえず”2”として処理を進める
    if line_spacing_factor == None:
            line_spacing_factor = 2
    
    for range_index in sorted_blocks:
        for block_index, block in sorted_blocks[range_index]:
            # 各ブロックを処理
            for line in block["lines"]:
                for span in line["spans"]:
                    bbox = span["bbox"]  # [x0, y0, x1, y1]
                    content = span["text"]
                    font = span["font"]
                    size = span["size"]
                    color = span["color"]
                    text_type = span.get("text_type", "text")  # デフォルトで"text"

                    # テキスト位置によるフィルタリング
                    if bbox[3] is not None and header_y is not None and bbox[3] < header_y:  # ヘッダの除外
                        continue
                    if bbox[1] is not None and footer_y is not None and bbox[1] > footer_y:  # フッタの除外
                        continue
                    if bbox[2] is not None and left_margin_x is not None and bbox[2] < left_margin_x:  # 左マージンの除外
                        continue
                    if bbox[0] is not None and right_margin_x is not None and bbox[0] > right_margin_x:  # 右マージンの除外
                        continue

                    # ひとまとまりの段落とみなす範囲の判定
                    #閾値の設定
                    line_threshold = size * line_spacing_factor  # フォントサイズに基づく閾値

                    #前の行との行間の大小を閾値と比較し、前の行の行末に改行を入れるか判断
                    if (
                        (previous_bbox and abs(bbox[3] - previous_bbox[3]) > line_threshold) or  # 行間が閾値よりも大
                        previous_text_type != text_type
                    ):
                        text += "\n\n"  # 新しい段落の開始とみなし、content（テキスト本体）を追加する前に改行を加える
                    else:
                        text += ""  # 同じ段落と見なして連結

                    # \nの直前にある半角スペースを除外
                    content = re.sub(r" \n", "\n", content)
                    content = re.sub(r"^ ", "", content)

                    # テキストを追加
                    text += content

                    # 状態を更新
                    previous_font = font
                    previous_size = size
                    previous_color = color
                    previous_bbox = bbox
                    previous_text_type = text_type

    # 最終的な改行の調整
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    return text

In [5]:
def main():
    # CSVファイルのパス
    #csv_path = 'data/Attrib_Analysis_Target_20250114_test.csv'  # CSVのパスを指定
    csv_path = 'data/Attrib_Analysis_Target_20250219_Denka.csv'  # CSVのパスを指定

    # 統合報告書がリストされたCSVを読み込む
    df_target = load_csv(csv_path)
    if df_target is None:
        print("Failed to load CSV. Exiting program.")
        logging.error("Failed to load CSV. Exiting program.")
        return

    # 必要な列名を指定
    folder_col = '格納フォルダ'
    file_col = '統合報告書PDF'
    #target_col = 'target_page'
    line_spacing_factor_col = '行間設定'

    # CSVに必要な列がなければエラー表示
    if folder_col not in df_target.columns or file_col not in df_target.columns not in df_target.columns:
        #print(f"Error: CSV file must contain '{folder_col}' and '{file_col}' columns")
        logging.error(f"Error: CSV file must contain '{folder_col}' and '{file_col}' columns")
        return

    # リストの企業ごとに、pdfのテキストデータを取得・保存
    #file_paths = []
    for index, row in df_target.iterrows():

        #================================================================
        #前処理（フォルダの確認・パス名の定義　等）
        folder_name = 'ir/2025/' + row[folder_col]
        file_name = row[file_col]
        line_spacing_factor = row[line_spacing_factor_col]

        info_file_path = folder_name + '/' + 'info'
        pdf_path = folder_name + "/" + file_name
        margin_path = info_file_path  + "/" + "margin.csv"
        multicolumn_path = info_file_path  + "/" + "multicolumn.txt"

        # パスが有効かチェック
        if pd.isna(folder_name) or pd.isna(file_name):
            print(f"Skipping row {index}: Folder or File is missing")
            logging.warning(f"Skipping row {index}: Folder or File is missing")
            file_path =''
            return
        #属性情報を格納するフォルダ（info）の有無を確認
        if not os.path.exists(info_file_path):
            logging.warning(f"Skipping row {index}: No info Folder")
            return

        #print(f"==== Processing {folder_name}/{file_name} ====")
        logging.info(f"==== Processing {folder_name}/{file_name} ====")

        # フィルタリング範囲の指定（適宜調整）
        # 初期化 (ファイルが存在しない場合もNoneになるようにする)
        header_y = None
        footer_y = None
        left_margin_x = None
        right_margin_x = None

        try:
            # margin.csvの読み込み
            df_margin = pd.read_csv(margin_path)
            
            # 変数への値の格納 (ブランクの場合はNoneに)
            header_y = df_margin.iloc[0]['header_y'] if 'header_y' in df_margin.columns and pd.notnull(df_margin.iloc[0]['header_y']) else None
            footer_y = df_margin.iloc[0]['footer_y'] if 'footer_y' in df_margin.columns and pd.notnull(df_margin.iloc[0]['footer_y']) else None
            left_margin_x = df_margin.iloc[0]['left_margin_x'] if 'left_margin_x' in df_margin.columns and pd.notnull(df_margin.iloc[0]['left_margin_x']) else None
            right_margin_x = df_margin.iloc[0]['right_margin_x'] if 'right_margin_x' in df_margin.columns and pd.notnull(df_margin.iloc[0]['right_margin_x']) else None
        
        except FileNotFoundError:
            # ファイルが存在しない場合、全ての変数をNoneのままにする
            logging.warning(f"File 'margin.csv' is missing, default values were applied.")
            pass
    
        # 結果を確認
        #print(f"header_y: {header_y}")
        #print(f"footer_y: {footer_y}")
        #print(f"left_margin_x: {left_margin_x}")
        #print(f"right_margin_x: {right_margin_x}")
        
        #段組み情報を設定
        # 段組みの閾値を x0 で設定
        column_thresholds = []  
        try:
            # ファイルを開いて読み込む
            with open(multicolumn_path, 'r', encoding='utf-8-sig') as mc_file:
                for line in mc_file:
                    # 各行をカンマで分割し、数値に変換してリストに格納
                    column_thresholds.extend([float(value) for value in line.strip().split(',') if value])
        except FileNotFoundError:
            logging.warning(f"File 'multicolumn.txt' is missing.")
            pass

        except ValueError:
            logging.warning(f"File 'multicolumn.txt' includes non-value data.")
            pass
        
        # 読み込んだ段組み閾値情報を確認
        #print(column_thresholds)
    
        # PDFファイルのオープン
        doc = fitz.open(pdf_path)
        
        # PDFからテキストを抽出し、クリーニング
        cleaned_text = ""
        
        for page_number, page in enumerate(doc, start=0):
            # ページ開始タグ
            cleaned_text += f"<page {page_number}>\n"
            # ページ内容（新しいx0_rangesによる処理）
            cleaned_text += clean_text_by_attributes(page, header_y, footer_y, left_margin_x, right_margin_x, column_thresholds, line_spacing_factor) + "\n"
            # ページ終了タグ
            cleaned_text += f"</page {page_number}>\n\n"
        
        # 出力ファイルパス
        output_file_path = pdf_path[:len(pdf_path)-4] + ".txt"
        
        # ファイルへの書き込み
        with open(output_file_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)
        
        logging.info(f"File '{output_file_path}' was saved.")
        #print(f"File '{output_file_path}' was saved.")


if __name__ == "__main__":
    main()

2025-02-19 15:56:32,816 - INFO - ==== Processing ir/2025/40610_デンカ/denkareport2024.pdf ====
2025-02-19 15:56:41,164 - INFO - File 'ir/2025/40610_デンカ/denkareport2024.txt' was saved.
